In [ ]:
'''
Here I was playing around with making my own custom model wtih skip connections between each convolution layer. 
I had to make sure the convolutions didn't affect the image size. Unfortunatly I haven't had much success. 

I call it "SkipNet" - feel free to use it if you want a 30% accurate bird classifier!

'''

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from datetime import datetime
#from ignite.metrics import Accuracy

In [2]:
# Hyperparameters
BATCH_SIZE = 100
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def Accuracy(output, labels):
    _, preds = torch.max(output, dim=1)

    return torch.sum(preds == labels).item() / len(preds)

In [4]:

'''
Data formatting:
pretrained models expect mini-batches of 3-channel RGB images of shape (N, 3, H, W), where
N is the number of images, H and W are expected to be at least 224
pixels. The images have to be loaded in to a range of [0, 1] and then
normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].

https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/
'''
data_dir = './birds_400'
trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std =[0.229, 0.224, 0.225])
])

train_ds = ImageFolder(f'{data_dir}/train', transform=trans)
test_ds  = ImageFolder(f'{data_dir}/test', transform=trans)
valid_ds = ImageFolder(f'{data_dir}/valid', transform=trans)

train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=3, pin_memory=True)
test_dl  = DataLoader(test_ds, BATCH_SIZE, shuffle=False, num_workers=3, pin_memory=True)
valid_dl = DataLoader(valid_ds, BATCH_SIZE, shuffle=False, num_workers=3, pin_memory=True)

In [5]:
train_ds[0][0].shape

torch.Size([3, 224, 224])

In [6]:
class SkipNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.c1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.c2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.c3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=3*112*112, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=num_classes)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        identity = x
        out = x
        for conv in [self.c1, self.c2, self.c3]:
            out = self.relu(conv(out)) + identity
            identity = out
        out = self.maxpool(out)
        out = torch.flatten(out, 1)
        out = self.relu(self.fc1(out))
        logits = self.fc2(out)
        return logits
        
        
        
    

In [7]:
model = SkipNet(num_classes=400)
#model = torch.load('skipnet.model')
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.1,
    patience=0,
    verbose=True
)

In [8]:
# torch.autograd.set_detect_anomaly(True)

In [12]:
# training
print("Beginning training at ", datetime.now())
history = []
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    for batch_idx, (images, labels) in enumerate(train_dl):
        print(f"epoch {epoch+1}: {batch_idx}/{len(train_dl)}\r", end="", flush=True)
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        pred = model(images)
        loss = loss_fn(pred, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    history.append(train_loss / (batch_idx + 1))
    print(f"Train loss: {history[-1]}", end=" ", flush=True)
    model.eval()
    with torch.no_grad():
        val_loss, val_acc = 0, 0
        for batch_idx, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)
            pred = model(images)
            loss = loss_fn(pred, labels)
            acc = Accuracy(pred, labels)
            val_loss += loss.item()
            val_acc += acc
        val_loss /= batch_idx + 1
        val_acc /= batch_idx + 1
    scheduler.step(val_acc / (batch_idx + 1))
    print(f"Val loss: {val_loss} Val Accuracy: {val_acc}", flush=True)

print(f"Validation loss: {val_loss}\nValidation Accuracy: {val_acc}")




Beginning training at  2022-03-27 14:37:07.277133
Train loss: 0.007110989387450127 Val loss: 5.896547245979309 Val Accuracy: 0.3175
Train loss: 0.007104849686833379 Val loss: 5.896923136711121 Val Accuracy: 0.3175
Train loss: 0.0070992718480227595 Val loss: 5.897259092330932 Val Accuracy: 0.3175
Train loss: 0.007092243109108906 Val loss: 5.897614192962647 Val Accuracy: 0.3175


KeyboardInterrupt: 

In [ ]:
'''
I only ran this a few times to show the loss was already at a local minimum, and that the accuracy plateaud at 0.3175.
I feel this is the limit of this model.

'''

In [ ]:
torch.save(model, "./skipnet.model")
print("Model has been saved.")